In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
offers = pd.read_csv("offerEvents.csv")
applications = pd.read_csv("applicationEvents.csv")
applications.head()

,case,event,startTime,completeTime,LoanGoal,ApplicationType,RequestedAmount,Action,FirstWithdrawalAmount,NumberOfTerms,Accepted,OfferID,MonthlyCost,EventOrigin,EventID,Selected,CreditScore,OfferedAmount
0,Application_652823628,A_Create Application,2016/01/01 10:51:15.304,2016/01/01 10:51:15.304,Existing loan takeover,New credit,20000.0,Created,NaN,NaN,NaN,NaN,NaN,Application,Application_652823628,NaN,NaN,NaN
1,Application_652823628,A_Submitted,2016/01/01 10:51:15.352,2016/01/01 10:51:15.352,Existing loan takeover,New credit,20000.0,statechange,NaN,NaN,NaN,NaN,NaN,Application,ApplState_1582051990,NaN,NaN,NaN
2,Application_652823628,A_Submitted,2016/01/01 10:51:15.352,2016/01/01 10:51:15.352,Existing loan takeover,New credit,20000.0,statechange,NaN,NaN,NaN,NaN,NaN,Application,ApplState_1582051990,NaN,NaN,NaN
3,Application_652823628,A_Submitted,2016/01/01 10:51:15.352,2016/01/01 10:51:15.352,Existing loan takeover,New credit,20000.0,statechange,NaN,NaN,NaN,NaN,NaN,Application,ApplState_1582051990,NaN,NaN,NaN
4,Application_652823628,A_Submitted,2016/01/01 10:51:15.352,2016/01/01 10:51:15.352,Existing loan takeover,New credit,20000.0,statechange,NaN,NaN,NaN,NaN,NaN,Application,ApplState_1582051990,NaN,NaN,NaN


In [3]:
made_offer = applications[pd.notnull(applications.Selected)]
made_offer =  made_offer.loc[((made_offer.event =='A_Pending') | (made_offer.event =='A_Cancelled') |
                             (made_offer.event =='A_Denied')) & made_offer.Accepted].reset_index()
# the last requirements makes sure that we only use applications which are accepted (and thus for which an offer is made)
del made_offer['index']
made_offer.drop_duplicates(inplace = True)
made_offer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22630 entries, 0 to 34220
Data columns (total 18 columns):
case                     22630 non-null object
event                    22630 non-null object
startTime                22630 non-null object
completeTime             22630 non-null object
LoanGoal                 22630 non-null object
ApplicationType          22630 non-null object
RequestedAmount          22630 non-null float64
Action                   22630 non-null object
FirstWithdrawalAmount    22630 non-null float64
NumberOfTerms            22630 non-null float64
Accepted                 22630 non-null object
OfferID                  22630 non-null object
MonthlyCost              22630 non-null float64
EventOrigin              22630 non-null object
EventID                  22630 non-null object
Selected                 22630 non-null object
CreditScore              22630 non-null float64
OfferedAmount            22630 non-null float64
dtypes: float64(6), object(12)
memory u

In [4]:
made_offer[['Selected']] = made_offer[['Selected']].astype(bool)

In [5]:
made_offer = made_offer[['ApplicationType', 'LoanGoal', 'RequestedAmount', 'FirstWithdrawalAmount',
                         'NumberOfTerms', 'MonthlyCost', 'OfferedAmount', 'Selected']]
made_offer.head()

,ApplicationType,LoanGoal,RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,MonthlyCost,OfferedAmount,Selected
0,New credit,Existing loan takeover,20000.0,20000.0,44.0,498.29,20000.0,True
2,New credit,Home improvement,15000.0,15000.0,120.0,158.98,15000.0,True
4,New credit,Home improvement,35000.0,35000.0,120.0,366.08,35000.0,False
5,New credit,Existing loan takeover,13000.0,400.0,120.0,141.45,13000.0,False
6,New credit,Home improvement,15000.0,328.0,84.0,211.90,15000.0,True


Categorical attributes

In [6]:
loanGoal = tf.feature_column.categorical_column_with_vocabulary_list('LoanGoal',
                                                                           ['Existing loan takeover',
                                                                            'Home improvement', 'Car',
                                                                            'Other, see explanation',
                                                                            'Remaining debt home', 'Not speficied',
                                                                            'Unknown', 'Caravan / Camper', 'Tax payments',
                                                                            'Extra spending limit', 'Motorcycle', 'Boat',
                                                                            'Business goal'])
applicationType = tf.feature_column.categorical_column_with_vocabulary_list('ApplicationType',
                                                                           ['New credit', 'Limit raise'])

Numeric attributes

In [7]:
req_amount = tf.feature_column.numeric_column('RequestedAmount')
first_withdrawal_amount = tf.feature_column.numeric_column('FirstWithdrawalAmount')
number_of_terms = tf.feature_column.numeric_column('NumberOfTerms')
monthly_cost = tf.feature_column.numeric_column('MonthlyCost')
offered_amount = tf.feature_column.numeric_column('OfferedAmount')

In [8]:
feat_cols = [loanGoal, applicationType, req_amount, first_withdrawal_amount, number_of_terms,
            monthly_cost, offered_amount]

In [9]:
x_features = made_offer.drop('Selected', axis = 1)

In [10]:
pred_labels = made_offer['Selected']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x_features, pred_labels, test_size = 0.2)

In [12]:
input_f = tf.estimator.inputs.pandas_input_fn(x = x_train, y = y_train, batch_size = 100, 
                                              num_epochs = 1000, shuffle = True)

In [13]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols, n_classes = 2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\s158881\\AppData\\Local\\Temp\\tmpywksx8m3', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [14]:
model.train(input_fn = input_f, steps = int(1e6))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\s158881\AppData\Local\Temp\tmpywksx8m3\model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 478.15
INFO:tensorflow:loss = 16655.3, step = 101 (0.212 sec)
INFO:tensorflow:global_step/sec: 519.14
INFO:tensorflow:loss = 8957.68, step = 201 (0.193 sec)
INFO:tensorflow:global_step/sec: 530.153
INFO:tensorflow:loss = 14438.0, step = 301 (0.189 sec)
INFO:tensorflow:global_step/sec: 534.05
INFO:tensorflow:loss = 6077.17, step = 401 (0.187 sec)
INFO:tensorflow:global_step/sec: 538.727
INFO:tensorflow:loss = 2080.77, step = 501 (0.186 sec)
INFO:tensorflow:global_step/sec: 531.564
INFO:tensorflow:loss = 6363.5, step = 601 (0.188 sec)
INFO:tensorflow:global_step/sec: 541.676
INFO:tensorflow:loss = 11619.2, step = 701 (0.186 sec)
INFO:tensorflow:global_step/sec: 524.558
INFO:tensorflow:loss = 4869.13, step = 801 (0.190 sec)
INFO:tensorflow:global_step/sec: 535.839
INFO

INFO:tensorflow:global_step/sec: 464.808
INFO:tensorflow:loss = 590.968, step = 8401 (0.214 sec)
INFO:tensorflow:global_step/sec: 569.415
INFO:tensorflow:loss = 1386.2, step = 8501 (0.177 sec)
INFO:tensorflow:global_step/sec: 472.507
INFO:tensorflow:loss = 3587.12, step = 8601 (0.210 sec)
INFO:tensorflow:global_step/sec: 563.008
INFO:tensorflow:loss = 1811.14, step = 8701 (0.178 sec)
INFO:tensorflow:global_step/sec: 556.734
INFO:tensorflow:loss = 1634.78, step = 8801 (0.180 sec)
INFO:tensorflow:global_step/sec: 581.011
INFO:tensorflow:loss = 2425.52, step = 8901 (0.171 sec)
INFO:tensorflow:global_step/sec: 549.079
INFO:tensorflow:loss = 990.94, step = 9001 (0.182 sec)
INFO:tensorflow:global_step/sec: 407.278
INFO:tensorflow:loss = 2066.0, step = 9101 (0.245 sec)
INFO:tensorflow:global_step/sec: 464.081
INFO:tensorflow:loss = 2504.72, step = 9201 (0.216 sec)
INFO:tensorflow:global_step/sec: 524.553
INFO:tensorflow:loss = 1302.06, step = 9301 (0.191 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:global_step/sec: 547.584
INFO:tensorflow:loss = 2379.44, step = 16801 (0.183 sec)
INFO:tensorflow:global_step/sec: 458.891
INFO:tensorflow:loss = 2222.17, step = 16901 (0.219 sec)
INFO:tensorflow:global_step/sec: 492.309
INFO:tensorflow:loss = 726.96, step = 17001 (0.203 sec)
INFO:tensorflow:global_step/sec: 399.492
INFO:tensorflow:loss = 275.923, step = 17101 (0.249 sec)
INFO:tensorflow:global_step/sec: 472.673
INFO:tensorflow:loss = 2185.2, step = 17201 (0.213 sec)
INFO:tensorflow:global_step/sec: 414.513
INFO:tensorflow:loss = 706.033, step = 17301 (0.240 sec)
INFO:tensorflow:global_step/sec: 450.154
INFO:tensorflow:loss = 1044.2, step = 17401 (0.223 sec)
INFO:tensorflow:global_step/sec: 553.65
INFO:tensorflow:loss = 425.934, step = 17501 (0.181 sec)
INFO:tensorflow:global_step/sec: 558.289
INFO:tensorflow:loss = 1299.03, step = 17601 (0.180 sec)
INFO:tensorflow:global_step/sec: 489.876
INFO:tensorflow:loss = 2248.62, step = 17701 (0.204 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 401.967
INFO:tensorflow:loss = 1283.34, step = 25201 (0.251 sec)
INFO:tensorflow:global_step/sec: 527.329
INFO:tensorflow:loss = 1259.9, step = 25301 (0.187 sec)
INFO:tensorflow:global_step/sec: 509.894
INFO:tensorflow:loss = 545.562, step = 25401 (0.197 sec)
INFO:tensorflow:global_step/sec: 445.14
INFO:tensorflow:loss = 2290.34, step = 25501 (0.224 sec)
INFO:tensorflow:global_step/sec: 457.336
INFO:tensorflow:loss = 1193.14, step = 25601 (0.219 sec)
INFO:tensorflow:global_step/sec: 200.068
INFO:tensorflow:loss = 781.033, step = 25701 (0.500 sec)
INFO:tensorflow:global_step/sec: 271.559
INFO:tensorflow:loss = 1079.86, step = 25801 (0.368 sec)
INFO:tensorflow:global_step/sec: 366.729
INFO:tensorflow:loss = 2410.27, step = 25901 (0.274 sec)
INFO:tensorflow:global_step/sec: 290.084
INFO:tensorflow:loss = 1021.11, step = 26001 (0.344 sec)
INFO:tensorflow:global_step/sec: 495.953
INFO:tensorflow:loss = 2142.5, step = 26101 (0.202 sec)
INFO:tensorflow:global_

INFO:tensorflow:global_step/sec: 490.373
INFO:tensorflow:loss = 1486.36, step = 33601 (0.204 sec)
INFO:tensorflow:global_step/sec: 407.519
INFO:tensorflow:loss = 1801.78, step = 33701 (0.245 sec)
INFO:tensorflow:global_step/sec: 421.703
INFO:tensorflow:loss = 805.437, step = 33801 (0.237 sec)
INFO:tensorflow:global_step/sec: 502.594
INFO:tensorflow:loss = 328.645, step = 33901 (0.201 sec)
INFO:tensorflow:global_step/sec: 402.901
INFO:tensorflow:loss = 630.772, step = 34001 (0.246 sec)
INFO:tensorflow:global_step/sec: 423.912
INFO:tensorflow:loss = 3146.63, step = 34101 (0.236 sec)
INFO:tensorflow:global_step/sec: 551.378
INFO:tensorflow:loss = 1227.74, step = 34201 (0.183 sec)
INFO:tensorflow:global_step/sec: 484.375
INFO:tensorflow:loss = 1929.86, step = 34301 (0.207 sec)
INFO:tensorflow:global_step/sec: 407.902
INFO:tensorflow:loss = 1293.12, step = 34401 (0.248 sec)
INFO:tensorflow:global_step/sec: 408.72
INFO:tensorflow:loss = 2258.36, step = 34501 (0.242 sec)
INFO:tensorflow:globa

...

INFO:tensorflow:global_step/sec: 427.196
INFO:tensorflow:loss = 89.6124, step = 142301 (0.234 sec)
INFO:tensorflow:global_step/sec: 427.1
INFO:tensorflow:loss = 556.34, step = 142401 (0.234 sec)
INFO:tensorflow:global_step/sec: 425.812
INFO:tensorflow:loss = 268.031, step = 142501 (0.235 sec)
INFO:tensorflow:global_step/sec: 543.022
INFO:tensorflow:loss = 750.842, step = 142601 (0.185 sec)
INFO:tensorflow:global_step/sec: 423.425
INFO:tensorflow:loss = 209.563, step = 142701 (0.235 sec)
INFO:tensorflow:global_step/sec: 440.831
INFO:tensorflow:loss = 91.4519, step = 142801 (0.228 sec)
INFO:tensorflow:global_step/sec: 444.483
INFO:tensorflow:loss = 636.635, step = 142901 (0.225 sec)
INFO:tensorflow:global_step/sec: 424.434
INFO:tensorflow:loss = 639.573, step = 143001 (0.236 sec)
INFO:tensorflow:global_step/sec: 414.716
INFO:tensorflow:loss = 425.886, step = 143101 (0.240 sec)
INFO:tensorflow:global_step/sec: 433.596
INFO:tensorflow:loss = 125.253, step = 143201 (0.231 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 506.052
INFO:tensorflow:loss = 1220.16, step = 150601 (0.199 sec)
INFO:tensorflow:global_step/sec: 478.093
INFO:tensorflow:loss = 124.962, step = 150701 (0.209 sec)
INFO:tensorflow:global_step/sec: 427.353
INFO:tensorflow:loss = 356.993, step = 150801 (0.234 sec)
INFO:tensorflow:global_step/sec: 427.637
INFO:tensorflow:loss = 351.271, step = 150901 (0.233 sec)
INFO:tensorflow:global_step/sec: 418.631
INFO:tensorflow:loss = 148.189, step = 151001 (0.240 sec)
INFO:tensorflow:global_step/sec: 403.508
INFO:tensorflow:loss = 247.822, step = 151101 (0.248 sec)
INFO:tensorflow:global_step/sec: 470.025
INFO:tensorflow:loss = 539.127, step = 151201 (0.214 sec)
INFO:tensorflow:global_step/sec: 424.829
INFO:tensorflow:loss = 579.221, step = 151301 (0.235 sec)
INFO:tensorflow:global_step/sec: 506.051
INFO:tensorflow:loss = 211.9, step = 151401 (0.198 sec)
INFO:tensorflow:global_step/sec: 523.581
INFO:tensorflow:loss = 387.344, step = 151501 (0.191 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 548.738
INFO:tensorflow:loss = 869.948, step = 158901 (0.182 sec)
INFO:tensorflow:global_step/sec: 540.145
INFO:tensorflow:loss = 126.764, step = 159001 (0.186 sec)
INFO:tensorflow:global_step/sec: 543.64
INFO:tensorflow:loss = 456.415, step = 159101 (0.184 sec)
INFO:tensorflow:global_step/sec: 422.941
INFO:tensorflow:loss = 1015.89, step = 159201 (0.235 sec)
INFO:tensorflow:global_step/sec: 488.838
INFO:tensorflow:loss = 741.334, step = 159301 (0.206 sec)
INFO:tensorflow:global_step/sec: 530.145
INFO:tensorflow:loss = 397.384, step = 159401 (0.188 sec)
INFO:tensorflow:global_step/sec: 544.66
INFO:tensorflow:loss = 783.293, step = 159501 (0.184 sec)
INFO:tensorflow:global_step/sec: 427.486
INFO:tensorflow:loss = 591.903, step = 159601 (0.233 sec)
INFO:tensorflow:global_step/sec: 428.35
INFO:tensorflow:loss = 751.235, step = 159701 (0.233 sec)
INFO:tensorflow:global_step/sec: 433.531
INFO:tensorflow:loss = 688.653, step = 159801 (0.230 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 430.732
INFO:tensorflow:loss = 194.563, step = 167201 (0.232 sec)
INFO:tensorflow:global_step/sec: 432.812
INFO:tensorflow:loss = 964.31, step = 167301 (0.231 sec)
INFO:tensorflow:global_step/sec: 545.67
INFO:tensorflow:loss = 479.596, step = 167401 (0.185 sec)
INFO:tensorflow:global_step/sec: 391.122
INFO:tensorflow:loss = 494.775, step = 167501 (0.258 sec)
INFO:tensorflow:global_step/sec: 410.396
INFO:tensorflow:loss = 370.234, step = 167601 (0.241 sec)
INFO:tensorflow:global_step/sec: 558.277
INFO:tensorflow:loss = 752.788, step = 167701 (0.178 sec)
INFO:tensorflow:global_step/sec: 584.393
INFO:tensorflow:loss = 766.84, step = 167801 (0.172 sec)
INFO:tensorflow:global_step/sec: 422.989
INFO:tensorflow:loss = 744.493, step = 167901 (0.234 sec)
INFO:tensorflow:global_step/sec: 465.679
INFO:tensorflow:loss = 419.547, step = 168001 (0.216 sec)
INFO:tensorflow:global_step/sec: 529.256
INFO:tensorflow:loss = 556.266, step = 168101 (0.188 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 491.874
INFO:tensorflow:loss = 340.613, step = 175501 (0.204 sec)
INFO:tensorflow:global_step/sec: 511.184
INFO:tensorflow:loss = 361.802, step = 175601 (0.196 sec)
INFO:tensorflow:global_step/sec: 423.111
INFO:tensorflow:loss = 578.751, step = 175701 (0.236 sec)
INFO:tensorflow:global_step/sec: 478.141
INFO:tensorflow:loss = 283.574, step = 175801 (0.209 sec)
INFO:tensorflow:global_step/sec: 470.516
INFO:tensorflow:loss = 512.162, step = 175901 (0.213 sec)
INFO:tensorflow:global_step/sec: 438.519
INFO:tensorflow:loss = 453.881, step = 176001 (0.227 sec)
INFO:tensorflow:global_step/sec: 506.928
INFO:tensorflow:loss = 569.776, step = 176101 (0.198 sec)
INFO:tensorflow:global_step/sec: 530.098
INFO:tensorflow:loss = 789.838, step = 176201 (0.191 sec)
INFO:tensorflow:global_step/sec: 489.487
INFO:tensorflow:loss = 415.021, step = 176301 (0.203 sec)
INFO:tensorflow:global_step/sec: 470.252
INFO:tensorflow:loss = 631.632, step = 176401 (0.213 sec)
INFO:tenso

In [15]:
evaluation_model_function = tf.estimator.inputs.pandas_input_fn(x = x_test, y = y_test, batch_size = 100,
                                                               num_epochs = 1, shuffle = False)

In [16]:
results = model.evaluate(evaluation_model_function)
results

INFO:tensorflow:Starting evaluation at 2017-11-23-10:37:04
INFO:tensorflow:Restoring parameters from C:\Users\s158881\AppData\Local\Temp\tmpywksx8m3\model.ckpt-181040
INFO:tensorflow:Finished evaluation at 2017-11-23-10:37:05
INFO:tensorflow:Saving dict for global step 181040: accuracy = 0.650906, accuracy_baseline = 0.651127, auc = 0.525708, auc_precision_recall = 0.814797, average_loss = 6.12181, global_step = 181040, label/mean = 0.651127, loss = 602.333, prediction/mean = 0.99593


{'accuracy': 0.65090585,
 'accuracy_baseline': 0.6511268,
 'auc': 0.5257079,
 'auc_precision_recall': 0.81479716,
 'average_loss': 6.1218081,
 'global_step': 181040,
 'label/mean': 0.6511268,
 'loss': 602.33264,
 'prediction/mean': 0.99593019}